#TASK 1

Design tables for PhoneBook

In [3]:
import psycopg2

def connect():
    return psycopg2.connect(
        host="ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech",
        dbname="neondb",
        user="neondb_owner",
        password="npg_Um02TyoiHXhw"  
    )

def create_phonebook_table():
    conn = connect()
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS PhoneBook (
            id SERIAL PRIMARY KEY,
            first_name VARCHAR(100),
            last_name VARCHAR(100),
            phone VARCHAR(15) UNIQUE,
            email VARCHAR(100)
        );
    """)

    conn.commit()
    cur.close()
    conn.close()

create_phonebook_table()

#TASK 2

Implement two ways of inserting data into the PhoneBook

In [6]:
#1 way for csv file
import csv
import psycopg2

def insert_data_from_csv(csv_file):
    conn = connect()
    cur = conn.cursor()

    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            first_name, last_name, phone, email = row
            cur.execute("""
                INSERT INTO PhoneBook (first_name, last_name, phone, email)
                VALUES (%s, %s, %s, %s)
            """, (first_name, last_name, phone, email))

    conn.commit()
    cur.close()
    conn.close()

insert_data_from_csv('contacts.csv')

In [25]:
#2 way for console
import psycopg2

def insert_data_from_console():
    conn = connect()
    cur = conn.cursor()

    first_name = input("Enter first name: ")
    last_name = input("Enter last name: ")
    phone = input("Enter phone number: ")
    email = input("Enter email: ")

    cur.execute("""
        INSERT INTO PhoneBook (first_name, last_name, phone, email)
        VALUES (%s, %s, %s, %s)
    """, (first_name, last_name, phone, email))

    conn.commit()
    cur.close()
    conn.close()

insert_data_from_console()

#TASK 3

Implement updating data in the table (change user first name or phone)

In [ ]:
def connect():
    """
    Connects to the PostgreSQL database using psycopg2.
    :return: connection object
    """
    import psycopg2
    conn = psycopg2.connect(
        host="ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech",
        dbname="neondb",
        user="neondb_owner",
        password="npg_Um02TyoiHXhw"
    )
    return conn

def update_data(id, updates):
    """
    Updates multiple fields for a given user by their id.
    
    :param id: User ID for the record to update.
    :param updates: Dictionary with fields to update and their new values.
    """
    conn = connect()
    cur = conn.cursor()

    # List of valid fields that can be updated
    valid_fields = ['first_name', 'last_name', 'email', 'phone']
    
    # Validate the fields in the updates dictionary
    for field in updates:
        if field not in valid_fields:
            print(f"Field {field} is not valid!")
            return

    # Construct the dynamic query to update the fields
    set_clause = ", ".join([f"{field} = %s" for field in updates])
    query = f"UPDATE PhoneBook SET {set_clause} WHERE id = %s"
    
    # Execute the query with the values to update
    cur.execute(query, (*updates.values(), id))

    conn.commit()
    cur.close()
    conn.close()

def get_update_choice():
    """
    Prompts the user to choose which fields they want to update for a given user.
    """
    # Get the user ID
    id = input("Enter the user ID to update: ")

    # Prompt the user to choose which fields to update
    print("Select which fields you want to update:")
    print("1. First Name (first_name)")
    print("2. Last Name (last_name)")
    print("3. Phone Number (phone)")
    print("4. Email (email)")
    
    # Collect the user's choices
    choices = input("Enter the numbers of the fields you want to update (e.g., 1 2 3 4): ").split()

    # Dictionary to hold the fields to be updated
    updates = {}

    # Process each choice
    if '1' in choices:
        new_value = input("Enter new value for First Name: ")
        updates['first_name'] = new_value
    if '2' in choices:
        new_value = input("Enter new value for Last Name: ")
        updates['last_name'] = new_value
    if '3' in choices:
        new_value = input("Enter new value for Phone Number: ")
        updates['phone'] = new_value
    if '4' in choices:
        new_value = input("Enter new value for Email: ")
        updates['email'] = new_value

    # Call the update function with the user ID and updates dictionary
    if updates:
        update_data(id, updates)
        print(f"Data successfully updated for user with ID {id}!")
    else:
        print("No fields selected for update.")

# Example of how the script works
get_update_choice()


Select which fields you want to update:
1. First Name (first_name)
2. Last Name (last_name)
3. Phone Number (phone)
4. Email (email)
Data successfully updated for user with ID 5!


#TASK 4

Querying data from the tables (with different filters)

In [21]:
def connect():
    """
    Connects to the PostgreSQL database using psycopg2.
    :return: connection object
    """
    import psycopg2
    conn = psycopg2.connect(
        host="ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech",
        dbname="neondb",
        user="neondb_owner",
        password="npg_Um02TyoiHXhw"
    )
    return conn

def query_data(filters):
    """
    Queries the PhoneBook table with the provided filters.
    
    :param filters: Dictionary with fields to filter by and their values.
    """
    conn = connect()
    cur = conn.cursor()

    # Start the query with the SELECT statement
    query = "SELECT * FROM PhoneBook"
    
    # If filters are provided, add WHERE clause to the query
    if filters:
        # Construct the WHERE clause dynamically
        where_clause = " AND ".join([f"{field} LIKE %s" for field in filters])
        query += f" WHERE {where_clause}"

    # Execute the query with the filter values (add '%' to match first letter search)
    cur.execute(query, tuple([f"{value}%" if len(value) == 1 else value for value in filters.values()]))

    # Fetch and print the results
    rows = cur.fetchall()
    if rows:
        for row in rows:
            print(row)
    else:
        print("No records found with the given filters.")
    
    cur.close()
    conn.close()

def get_filter_input():
    """
    Prompts the user for filter options and queries the database based on those filters.
    """
    # Prompt user for input
    print("You can filter by the following fields:")
    print("1. First Name (first_name)")
    print("2. Last Name (last_name)")
    print("3. Phone Number (phone)")
    print("4. Email (email)")

    filters = {}

    # Collect user inputs for filters
    if input("Do you want to filter by First Name? (y/n): ").lower() == 'y':
        first_name = input("Enter First Name or First letter: ")
        filters['first_name'] = first_name
    if input("Do you want to filter by Last Name? (y/n): ").lower() == 'y':
        last_name = input("Enter Last Name or First letter: ")
        filters['last_name'] = last_name
    if input("Do you want to filter by Phone Number? (y/n): ").lower() == 'y':
        phone = input("Enter Phone Number: ")
        filters['phone'] = phone
    if input("Do you want to filter by Email? (y/n): ").lower() == 'y':
        email = input("Enter Email: ")
        filters['email'] = email

    # Query the data based on user input
    if filters:
        query_data(filters)
    else:
        print("No filters provided, showing all records.")
        query_data({})

# Example of how the script works
get_filter_input()

You can filter by the following fields:
1. First Name (first_name)
2. Last Name (last_name)
3. Phone Number (phone)
4. Email (email)
(5, 'Simon', 'Say', '9876543210', 'Simoncmon@gmail.com')
(7, 'Simon', 'Xeleon', '123123123', 'SimonBBB@gmail.com')


#TASK 5

Implement deleting data from tables by username of phone

In [24]:
import psycopg2

def connect():
    """
    Connects to the PostgreSQL database using psycopg2.
    :return: connection object
    """
    conn = psycopg2.connect(
        host="ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech",
        dbname="neondb",
        user="neondb_owner",
        password="npg_Um02TyoiHXhw"
    )
    return conn

def delete_data(full_name=None, phone=None):
    """
    Deletes data from the PhoneBook table based on full name (first name and last name combined) or phone number.
    
    :param full_name: The full name (first and last name combined) of the person to delete.
    :param phone: The phone number of the person to delete.
    """
    conn = connect()
    cur = conn.cursor()

    # Initialize query and parameters
    query = "DELETE FROM PhoneBook WHERE"
    conditions = []
    params = []

    # If full name is provided, split it into first and last name
    if full_name:
        first_name, last_name = full_name.split(maxsplit=1)  # Split by first space, assuming two parts
        conditions.append("first_name = %s AND last_name = %s")
        params.extend([first_name, last_name])

    if phone:
        conditions.append("phone = %s")
        params.append(phone)

    # If conditions exist, add them to the query
    if conditions:
        query += " " + " AND ".join(conditions)  # Make sure there's a space after WHERE

    # Execute the query
    cur.execute(query, tuple(params))

    # Commit the transaction
    conn.commit()

    # Check how many rows were deleted
    print(f"Rows deleted: {cur.rowcount}")
    
    # Close the cursor and connection
    cur.close()
    conn.close()

# Get user input through console
print("Enter the details of the person you want to delete:")

full_name = input("Enter full name (first and last name) (leave empty to skip): ")
phone = input("Enter phone number (leave empty to skip): ")

# Call delete_data based on provided inputs
delete_data(full_name=full_name or None, phone=phone or None)

Enter the details of the person you want to delete:
Rows deleted: 1
